In [28]:
# Data import
import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)
  

In [ ]:
# Question 1
# Create the model
model = pe.ConcreteModel()

# Define sets
model.F = pe.Set(initialize=factories)
model.R = pe.Set(initialize=rebars.keys())
model.L = pe.Set(initialize=longbars.keys())
model.A = pe.Set(initialize=customers)
model.T = pe.Set(initialize=periods)


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model.x = pe.Var(model.R, model.L_N_F, model.T, domain=pe.NonNegativeIntegers)
model.y = pe.Var(model.L_N_F, model.T, domain=pe.Binary)
model.z = pe.Var(model.F, model.A, model.T, domain=pe.Binary)

# Constraints using ConstraintList and for loops
# 1. Cutting constraints
model.cutting_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for l in model.L:
            for n in possible_longbars[l][f]:
                expression = sum(model.x[r, l, n, f, t] * w_r[r] for r in model.R) <= model.y[l, n, f, t] * w_l[l]
                model.cutting_cnstr.add(expression)

# 2. Production capacity
model.capacity_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        expression = sum(model.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model.L) * density * area <= production_capacity[f]
        model.capacity_cnstr.add(expression)

# 3. Demand satisfaction
model.demand_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for r in model.R:
            expression = sum(model.x[r, l, n, f, t] for n in possible_longbars[l][f] for l in model.L) >= sum(model.z[f, a, t] * demand[r][a][t-1] for a in model.A)
            model.demand_cnstr.add(expression)

# 4. Single sourcing
model.single_sourcing_cnstr = pe.ConstraintList()
for a in model.A:
    for t in model.T:
        expression = sum(model.z[f, a, t] for f in model.F) == 1
        model.single_sourcing_cnstr.add(expression)


# Add the production triggers customer assignment constraint to the model
model.prod_triggers_customer_assignment = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        # Sum over all r, l, n as specified
        expression = sum(model.y[ l, n, f, t]
                         for l in model.L 
                         for n in possible_longbars[l][f]) <= M * sum(model.z[f, a, t] for a in model.A)
        
        # Add the constraint to the model
        model.prod_triggers_customer_assignment.add(expression)


# Objective function
objExpr = sum(
    model.z[f, a, t] * (
        fixed_cost[f] + 
        v * distance[a][f] * 
        sum(demand[r][a][t-1] * w_r[r] for r in model.R) * 
        density * area
    )
    for f in model.F
    for a in model.A
    for t in model.T
)
model.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True, options={'TimeLimit':36})

# Display results
print(f"\nSolver Status: {result.solver.status}")
print(f"Termination Condition: {result.solver.termination_condition}")
if result.solver.status == pe.SolverStatus.ok and result.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model.F:
        for a in model.A:
            for t in model.T:
                if pe.value(model.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model.R:
        for l in model.L:
            for f in model.F:
                for n in possible_longbars[l][f]:
                    for t in model.T:
                        if pe.value(model.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model.L:
        for f in model.F:
            for n in possible_longbars[l][f]:
                for t in model.T:
                    if pe.value(model.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model.y[l, n, f, t]):.0f}")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2618603
Academic license - for non-commercial use only - expires 2026-02-05
Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpgkey_3pq.pyomo.lp
Reading time = 0.07 seconds
x1: 2044 rows, 7904 columns, 16928 nonzeros
Set parameter TimeLimit to value 36
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  36

Optimize a model with 2044 rows, 7904 columns and 16928 nonzeros
Model fingerprint: 0x518e30c2
Variable types: 0 continuous, 7904 integer (2048 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+02]
  Objective range  [2e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 721 rows and 2816 columns
Presolve time: 1.25s
Presolved: 1323 rows, 5088 co

In [24]:
f = "South Wales"
for l in model.L:
    for n in possible_longbars[l][f]:
        if pe.value(model.y[ l, n, f, 1]) > 0:
            
           print(f"{l}, {n}, {pe.value(model.y[l, n, f, 1]):.0f}")

In [26]:
# Analysis of production and demand

total_demand_per_period = {1: 0, 2: 0, 3: 0, 4: 0}
for r in rebars:
    for customer in demand[r]:
        for t in range(4):  # For each period
            # Demand in tonnes for each rebar type, customer, and period
            demand_in_tonnes = demand[r][customer][t] * w_r[r] *density*area
            total_demand_per_period[t + 1] += demand_in_tonnes

# Initialize a dictionary to store production data
factory_production = {f: {t: 0 for t in range(1, 5)} for f in ["IJmuiden", "Segal", "South Wales"]}
total_production_per_period = {t: 0 for t in range(1, 5)}  # Total production for each period

# Assuming 'demand' is defined as in your previous data structure
# demand = { ... }  # Make sure demand is properly initialized

# Extract production values from the model
for f in factory_production:
    for t in factory_production[f]:
        total_tonnes = sum(
            pe.value(model.y[l, n, f, t]) * w_l[l] * density * area
            for l in model.L for n in possible_longbars[l][f]
            if pe.value(model.y[l, n, f, t]) > 0.5
        )
        factory_production[f][t] = round(total_tonnes, 2)  # Round to 2 decimals
        total_production_per_period[t] += total_tonnes  # Update total production per period


# Print formatted table
print("Factory Production (Tonnes per Period)")
print("-" * 80)
print(f"{'Factory':<15} {'P1':<10} {'P2':<10} {'P3':<10} {'P4':<10} ")
print("-" * 80)

# Print the factory production values
for f, periods in factory_production.items():
    row = f"{f:<15}"
    for t in range(1, 5):
        row += f"{periods[t]:<10}"
    print(row)

# Print the row for total production across all factories
row = f"{'Total Production':<15}"
for t in range(1, 5):
    row += f"{total_production_per_period[t]:<10}"
row += f"{round(sum(total_production_per_period.values()), 2):<15.2f}"  # Rounded total production across all periods
print(row)

# Print the row for total production across all factories
row = f"{'Total Demand':<15}"
for t in total_demand_per_period:
    row += f"{round(total_demand_per_period[t],2):<10}"
print(row)

print()
# Cost per period

total_cost_per_period = {t: 0 for t in model.T}  # Dictionary to store total cost for each period

# Loop through each period, customer, and factory to calculate the total cost per period
for t in model.T:
    period_cost = 0  # Initialize the cost for the current period
    for f in model.F:
        for a in model.A:
            # Calculate the cost of transportation and fixed costs for each factory, customer, and period
            period_cost += pe.value(model.z[f, a, t]) * (
                fixed_cost[f] +  v * distance[a][f] * sum(demand[r][a][t-1] * w_r[r] for r in model.R) * density * area
            )
    # Store the total cost for this period
    total_cost_per_period[t] = period_cost

# Print the total cost per period
for t in total_cost_per_period:
    print(f"Total cost for Period {t}: €{total_cost_per_period[t]:.2f}")


Factory Production (Tonnes per Period)
--------------------------------------------------------------------------------
Factory         P1         P2         P3         P4         
--------------------------------------------------------------------------------
IJmuiden       7.57      11.84     11.96     11.96     
Segal          9.98      9.98      9.98      9.98      
South Wales    0         11.84     14.72     7.99      
Total Production17.54742341485097533.65259285039912436.6572886406133329.926770070533507117.78         
Total Demand   15.48     23.73     23.06     25.02     

Total cost for Period 1: €2836.52
Total cost for Period 2: €4677.77
Total cost for Period 3: €4255.74
Total cost for Period 4: €5065.85
